A first attempt at getting a working LGBM Model running

It works, but the scores are terrible

The best model is pickled to 'model/best_LGBM.pkl'

# IMPORTS

In [68]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import numpy as np
from lightgbm import LGBMClassifier
import re
import gc

# READ DATA

In [69]:
USE_SAVED_DATA = True

if USE_SAVED_DATA:
    print ('Using saved data')
    df = pd.read_csv('testdata/hdcr10k.csv')
else:
    print ('Using FULL data')
    df = pd.read_csv('../data/cleaned/hcdr_FULL.csv',nrows=10000)
    df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    df.to_csv('testdata/hdcr10k.csv', index=False)
df.shape

Using saved data


(10000, 798)

In [70]:
print (df.isna().sum().sum())

2058717


In [71]:
# No need to do this with LGBM
# df.fillna(0,inplace=True)

# X and y

In [72]:
X= df.drop('TARGET',axis=1)
y= df['TARGET']

# SPLIT TEST/TRAIN

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)



# GRIDSEARCHCV

In [74]:
from sklearn.model_selection import GridSearchCV

folds = KFold(n_splits= 5, shuffle=True, random_state=1001)

params = {
    'n_estimators': [100],
    'learning_rate':[0.1,0.02],
    'num_leaves':[34,40],
    'colsample_bytree':[0.9497036],
    'subsample':[0.8715623],
    'max_depth':[5, 8],
    'reg_alpha':[0.041545473],
    'reg_lambda':[0.0735294],
    'min_split_gain':[0.0222415],
    'min_child_weight':[39.3259775],
}

grid = GridSearchCV(LGBMClassifier(random_state=0), params, scoring='r2', cv=folds, verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=-0.085 total time=   1.1s
[CV 2/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=-0.108 total time=   1.2s
[CV 3/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=-0.111 total time=   1.2s
[CV 4/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34,

GridSearchCV(cv=KFold(n_splits=5, random_state=1001, shuffle=True),
             estimator=LGBMClassifier(random_state=0),
             param_grid={'colsample_bytree': [0.9497036],
                         'learning_rate': [0.1, 0.02], 'max_depth': [5, 8],
                         'min_child_weight': [39.3259775],
                         'min_split_gain': [0.0222415], 'n_estimators': [100],
                         'num_leaves': [34, 40], 'reg_alpha': [0.041545473],
                         'reg_lambda': [0.0735294], 'subsample': [0.8715623]},
             scoring='r2', verbose=3)

# RESULTS

In [75]:
print (grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

-0.08623363498400397
LGBMClassifier(colsample_bytree=0.9497036, learning_rate=0.02, max_depth=5,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               num_leaves=34, random_state=0, reg_alpha=0.041545473,
               reg_lambda=0.0735294, subsample=0.8715623)
{'colsample_bytree': 0.9497036, 'learning_rate': 0.02, 'max_depth': 5, 'min_child_weight': 39.3259775, 'min_split_gain': 0.0222415, 'n_estimators': 100, 'num_leaves': 34, 'reg_alpha': 0.041545473, 'reg_lambda': 0.0735294, 'subsample': 0.8715623}


# EXTRACT BEST MODEL

In [76]:
from sklearn.metrics import r2_score

best_model = grid.best_estimator_

best_model.fit(X_train, y_train)
y_predict = best_model.predict(X_test)
print ("R2 score:",r2_score(y_test,y_predict))

R2 score: -0.07526881720430123


# SAVE MODEL

In [77]:
import joblib

joblib.dump(best_model, 'model/best_LGBM.pkl')

['model/best_LGBM.pkl']